In [1]:
from imports import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from hit_makers.biasgan_hit_maker import BiasganHitMakerRace
from hit_makers.biasgan_hit_maker import task_to_class

In [25]:
directory = "biasgan/mturk_data"
task1 = "Task1_Race_Classification"
task2 = "Task2_Real_Fake_Classification"
task3 = "Task3_Quality_Ranking"

task = task3
# NOTE(ethan): task 3 data folder seems incorrect right now
maker = task_to_class[task]()

task_folder = goat.pjoin(directory, task)
print(task_folder)

# from hit_makers.biasgan_hit_maker import get_processed_data_from_task_folder
pdata = maker.get_processed_data_from_media_folder(task_folder)
# pprint.pprint(pdata)

biasgan/mturk_data/Task3_Quality_Ranking


In [26]:
# hit

In [27]:
# now, break the save a bunch of hits (e.g., 40)...
# 100 images per HIT
# chunk with goat
size_of_chunk = 100

query_folder_to_query_examples = pdata["query_folder_to_query_examples"]
for query_folder in query_folder_to_query_examples:
    query_examples = query_folder_to_query_examples[query_folder]
    random.shuffle(query_examples)
    query_examples_chunks = goat.utils.get_chunks(query_examples, size_of_chunk=size_of_chunk)
    for hit_idx, qes in enumerate(query_examples_chunks):
        examples = pdata["examples"]
        gt_query_examples = random.sample(pdata["gt_query_examples"], k=10)
        hit = maker.get_hit(task=task, 
                            task_question=pdata["task_question"], 
                            examples=examples, 
                            gt_query_examples=gt_query_examples, 
                            query_examples=qes)
        hit_name = "{}-{}-{:06d}".format(task, query_folder, hit_idx)
#         print(hit_name)
        maker.save(hit_name, hit)

In [46]:
# "/shared/ethanweber/anno/static/data/hits"
interface_endpoint = "https://biasgan.ethanweber.me/interface"
hit_names = os.listdir("/shared/ethanweber/anno/static/data/hits/")
external_urls = []
for hit_name in sorted(hit_names):
    external_urls.append(os.path.join(interface_endpoint, hit_name.replace(".json", "")))
# external_urls

In [59]:
dolors_per_hour = 9
minutes_per_hit = 5
cost_per_hit = (minutes_per_hit / 60) * 9
num_hits = 100
mturk_tax = 1.4
num_assignments_per_hit = 3
cost = cost_per_hit * num_hits * num_assignments_per_hit * mturk_tax
print(cost_per_hit)
print(cost)

0.75
315.0


In [52]:
# 100 * 5.4

In [53]:
# hit_names

In [54]:
# examples = [example] * 10
# hit = maker.get_hit(examples, examples, examples)
# pprint.pprint(hit)
# hit_name = "hit_example"
# maker.save(hit_name, hit)

In [43]:
from requester import Requester

In [63]:
use_sandbox=False
DATABASE_FILENAME = "/shared/ethanweber/anno/mturk_database.pkl"
CREDS_FILENAME = "/shared/ethanweber/anno/mturk_creds.json"

requester = Requester(use_sandbox=use_sandbox,
                      database_filename=DATABASE_FILENAME,
                      creds_filename=CREDS_FILENAME)
requester.show_account_balance()

No database found. Initializing it to:
{'production': {}, 'sandbox': {}}
87.10


In [64]:
lifetime_hours = 1
per_hit_minutes = 15
cost_per_hit = '0.5'
days_until_autoapprove = 2
estimated_time_in_minutes = 5
task_attributes = {
    # Number of assignments per task.
    'MaxAssignments': 3,
    # How long the task will be available on MTurk
    'LifetimeInSeconds': 60 * 60 * lifetime_hours,
    # How long Workers have to complete each item
    'AssignmentDurationInSeconds': 60 * per_hit_minutes,
    # The reward you will offer Workers for each response
    'Reward': cost_per_hit,
    'Title': 'Identify the race of the person(s) in the image. (~{} min)'.format(estimated_time_in_minutes),
    'Keywords': 'image, choice, computer vison, selection',
    'Description': 'Identify the race of the person(s) in the image.',
    'AutoApprovalDelayInSeconds': days_until_autoapprove * (60*60*24)
}
requester.set_task_attributes(task_attributes)

In [66]:
requester.get_worker_requirements()

[{'QualificationTypeId': '2F1QJWKUDD8XADTFD2Q0G6UTO95ALH',
  'Comparator': 'Exists'}]

In [70]:
task1_external_urls = ["https://biasgan.ethanweber.me/interface/Task1_Race_Classification-black20_white80-000000"]


In [71]:
# https://biasgan.ethanweber.me/interface/Task1_Race_Classification-black20_white80-000000
# https://biasgan.ethanweber.me/interface/Task2_Real_Fake_Classification-black20_white80-000000
# https://biasgan.ethanweber.me/interface/Task3_Quality_Ranking-black20_white80-000000

In [73]:
hit_ids = requester.submit_hits_from_external_urls(task1_external_urls)

100%|██████████| 1/1 [00:00<00:00,  1.09it/s]


In [74]:
hit_ids

['3E22YV8GG10M66IRAKTYNDAKBAHPN5']

In [76]:
hit_ids = [v["hit_id"] for v in requester.database["production"].values()]

In [78]:
hit_ids

['3E22YV8GG10M66IRAKTYNDAKBAHPN5']

In [80]:
sorted(requester.database["production"].keys())

['3E22YV8GG10M66IRAKTYNDAKBAHPN5']

In [75]:
for hit_id in hit_ids:
    print(requester.get_url_to_view_hit(hit_id))

https://www.mturk.com/mturk/preview?groupId=3QA1H4GB4RMS7Q0NUEN7IBA9B623M8


In [83]:
requester.update_database_with_responses()


100%|██████████| 1/1 [00:00<00:00,  3.20it/s]

No results.


In [84]:
# responses = requester.get_responses_from_database()
# requester.save_all_responses_to_files(responses)

In [88]:
for hit_id in hit_ids:
    assignments_list = requester.client.list_assignments_for_hit(
                    HITId=hit_id,
                    MaxResults=100
                )
    print(assignments_list)
    print()

{'NumResults': 0, 'Assignments': [], 'ResponseMetadata': {'RequestId': '0adadf5f-fb47-473c-a9fd-7c3736883923', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '0adadf5f-fb47-473c-a9fd-7c3736883923', 'content-type': 'application/x-amz-json-1.1', 'content-length': '33', 'date': 'Mon, 28 Feb 2022 04:59:47 GMT'}, 'RetryAttempts': 0}}



In [313]:
# for hit_id in hit_ids:
#     print(requester.client.get_hit(HITId=hit_id))

In [86]:
# requester.get_mean_time_from_resonses(responses)

In [244]:
# stop hits
import datetime
for hit_id in hit_ids:
#     print('HITId:', hit_id)
    # Get HIT status
    status = requester.client.get_hit(HITId=hit_id)['HIT']['HITStatus']
    print('HITStatus:', status)
    # If HIT is active then set it to expire immediately
    if status=='Assignable':
        response = requester.client.update_expiration_for_hit(
            HITId=hit_id,
            ExpireAt=datetime.datetime(2015, 1, 1)
        )
        print(hit_id)